# Data Loading, Storage, and File Formats

Data output and data input:
- text files or other more effient on-disk formats  
+ database  
* web APIs  

## 1. Reading and Writing data in text format  

***read_csv*** and ***read_table*** are the most used ones to convert data into a **Dataframe**  

Parsing functions:
>read_csv  
>read_table  
>read_excel  
>read_html  
>read_json  
>read_sql

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('examples/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [22]:
df = pd.read_table('examples/ex1.csv',sep=',')
df
#also could use read_table and specified the delimiter

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


1.1 Assign **column names**:

In [18]:
pd.read_csv('examples/ex2.csv', header=None)
#default names

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [19]:
pd.read_csv('examples/ex2.csv', names=['a','b','c','d','message'])
#specify yourself

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


1.2 Columns to be the **index** of the returned DataFrame:

In [20]:
names=['a','b','c','d','message']
pd.read_csv('examples/ex2.csv',names=names, index_col='message')
#if u want a hierarchical index, pass a list of column numbers or names

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


1.3 Cases a table dont have a **fixed delimiter**:

In [23]:
list(open('examples/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

1.4 Use **\s+** to sep a variable amount of whitespace

In [24]:
result = pd.read_table('examples/ex3.txt', sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


1.5 Using **skiprows** to skip rows:

In [25]:
pd.read_csv('examples/ex4.csv', skiprows=[0,2,3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


1.6 Handling **missing values**:

In [27]:
result = pd.read_csv('examples/ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


The **na_values** option cat either a list or set of strings ot consider missing values

In [29]:
sentinels = {'message':['foo', 'NA'], 'something':['two']}
pd.read_csv('examples/ex5.csv', na_values = sentinels)
#Different NA sentinels can be specified for each column in a dict

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


### Reading Text Files in *Pieces*

1.7 When we want to read in **a small piece** of a file or **iterate** through smaller chunks of the file

In [59]:
result = pd.read_csv('examples/ex6.csv')
result.tail()

,one,two,three,four,key
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G
9999,-0.096376,-1.012999,-0.657431,-0.573315,0


Specify with **nrows**:

In [31]:
pd.read_csv('examples/ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


Specify a chunksize as a number of rows and returns a TextParser object, then u can **iterate** it:

In [44]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)

In [45]:
tot = pd.Series([])

In [58]:
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(),fill_value=0)
tot = tot.sort_values(ascending=False)
tot[:5]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
dtype: float64

### Writing Data to Text Format

1.8 Using **to_csv** method to write a comma-separated file by default:

In [60]:
import sys
data = pd.read_csv('examples/ex5.csv')

In [61]:
data.to_csv(sys.stdout)

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [64]:
data.to_csv(sys.stdout, sep='|', na_rep='Null', index=False, header=False, columns=['a','b','c'])
#sep to change the delimiter
#na_rep to replce the NA
#labels can be disabled
#columns to choose the columns u want to write(a subset)

1|2|3.0
5|6|Null
9|10|11.0


### Working with Delimited Formats

1.9 When file have one or more malformed lines that trip up **read_table**, use **import csv**:

In [65]:
import csv

In [69]:
pd.read_table('examples/ex7.csv')

,"a,""b"",""c"""
0,"1,""2"",""3"""
1,"1,""2"",""3"""


In [66]:
with open('examples/ex7.csv') as f:
    lines = list(csv.reader(f))
#read the file into a list of lines

In [70]:
for line in lines:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [71]:
header, values = lines[0], lines[1:]

In [78]:
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

In [74]:
pd.DataFrame(data_dict)

,a,b,c
0,1,2,3
1,1,2,3
